In [50]:
import urllib3
import json
import sys

In [51]:
urllib3.disable_warnings()
https = urllib3.PoolManager()
main_url_ebi = 'https://www.ebi.ac.uk/biosamples/samples'

In [122]:
class Api_client:
    
    list_of_status = {
            200: 'The response was ok.',
            201: 'Resource was created',
            400: 'The request was malformed. Change the request',
            401: 'Unauthorized',
            403: 'Forbidden for this request',
            404: 'Resource not found',
            405: 'Method not allowed'
        }
    
    @classmethod
    def control_errors_from_request(cls, status):
        """
        Control de status errors requested made to EBI API 
        """
        to_continue, return_result = True, None
        status = int(status)
        if status != 200:
            to_continue = False
        return_result = cls.list_of_status.get(status, 'Wrong Status')
        return to_continue, return_result
    
    
    @classmethod
    def get_method_client_method(cls, url_formed):
        """
        Reusable method GET
        :return tuple. if is ok the response. If the response is ok will return data as a Json
        otherwise will return the message according to Status code. 
        """
        request_url = https.request('GET', url_formed)
        response_is_ok, response = cls.control_errors_from_request(request_url.status)
        if response_is_ok:
            response = json.loads(request_url.data.decode('utf-8'))
        return response_is_ok, response

    
    @classmethod
    def get_accessions_by_filter(cls, attribute, value):
        """
        : return the name of accession. 
        : IF does not exist print the list_of_status text
        """
        # Transform the value and attribute.
        # If the value is full of spaces do not include the value in the URL
        attribute = attribute.replace(' ','+')
        value = value.replace(' ', '+') if not value.isspace() else None
        url_formed = main_url_ebi+'?filter=attr:{attribute}:{value}'.format(attribute=attribute, value=value ) if value else main_url_ebi+'?filter=attr:{attribute}'.format(attribute=attribute)
        response_is_ok, data = cls.get_method_client_method(url_formed)
        response = data
        if response_is_ok:
            _embedded = data.get('_embedded')
            if (_embedded is not None):
                samples = _embedded.get('samples')
                if samples is not None:
                    response = ''
                    for sample in samples:
                        response = response + sample.get('accession') + ', '
            else:
                response = ' No Samples found'
                    
        return response
    
    
    @classmethod
    def get_name_by_accession(cls, accession):
        """
        : return the name of accession. 
        : IF does not exist print the list_of_status text
        """
        url_formed = main_url_ebi+'/{accession}'.format(accession=accession)
        response_is_ok, data = cls.get_method_client_method(url_formed)
        response = data
        if response_is_ok:
            if (data.get('name') is not None):
                response = data.get('name')
        return response
    
    
    @classmethod
    def total_number_of_samples(cls):
        """
        :return -1 if any error, other case it is ok
        """
        url_formed = main_url_ebi
        response = -1
        response_is_ok, data = cls.get_method_client_method(url_formed)
        if response_is_ok:
            if (data.get('page') is not None) and (data.get('page').get('totalElements')):
                response = data.get('page').get('totalElements')
        return response
    


class Application:
    
    def __init__(self):
        pass
    
    def close():
        """
        Function to exit the Application.
        """
        sys.exit()
    
    def get_name_by_accession():
        accession = input("Accession: ")
        return Api_client.get_name_by_accession(accession)
    
    def get_accessions_by_filter():
        attribute = input("Attribute: ")
        value = input("Value: ")
        return Api_client.get_accessions_by_filter(attribute, value)
        
    
    def get_examples_queries_results():
        given_accession = """
        Accession    |  Name
        SAMEA1507265 |  source GSM1000387 1
        SAMD00000328 |  SAMD00000328
        SAMEA1481290 |  source GSM898528 1
        """
        
        given_attribute_query = """
        Attribute    |  Value.        | Results
        organism     |  homo sapiens  | SAMN08025863, SAMN08025864
        organism     |  Homo Sapeins  | No samples found
        description title |  BioSample1 | SAMN08025863
        INSDC last update |           | SAMD00000001, SAMD00000006, SAMD00000007, SAMD00000008, SAMD00000009, SAMD00000010, SAMD00000011, SAMD00000012, SAMD00000014, SAMD00000021, SAMD00000325, SAMD00000326, SAMD00000327, SAMD00000328, SAMD00000330, SAMD00000331, SAMD00000332, SAMD00000333, SAMD00000334, SAMD00000341
        """
        
        print(given_accession, given_attribute_query)
    
    MENU_OPTIONS = {
        1: { 'title': 'Get total number of samples', 'function': Api_client.total_number_of_samples, 'response': 'Total number of samples: ' },
        2: { 'title': 'Get Name for an given Accession', 'function': get_name_by_accession, 'response': 'Name of the given accession: '},
        3: { 'title': 'Get Filtered Accessions for an given Attribute and Value', 'function': get_accessions_by_filter, 'response': 'Accessions: '},
        4: { 'title': 'Examples of such queries and results', 'function': get_examples_queries_results, 'response': ''},
        5: {'title':'Exit', 'function': close}
    }
    
    @classmethod
    def print_menu_options(cls):
        print('Choose a menu')
        # Print menu options
        for key, value in cls.MENU_OPTIONS.items():
            print(key, value.get('title'))
    
    @classmethod
    def get_selected_menu_item(cls):
        """
        :return a tuple, function_to_call and the text for print the response
        """
        # Read input from
        try:
            menu = int(input("Option: "))
            choosen_option = cls.MENU_OPTIONS.get(menu)
            function_to_call = choosen_option.get('function')
            response_text = choosen_option.get('response')
            return function_to_call, response_text
        except ValueError as e:
            raise Exception('Invalid Option. ')
    
    @classmethod
    def call_function_get_response(cls, function_to_call, response_text):
        result = function_to_call()
        print(response_text, result, '\n')
"""

if __name__ == "__main__":
    while(True):
        try:
            Application.print_menu_options()
            function_to_call, response_text = Application.get_selected_menu_item()
            Application.call_function_get_response(function_to_call, response_text)
        except Exception as e:
            print('Error in API Client EBI, {} \n\r'.format(e))

"""
if __name__ == "__main__":
    Application.print_menu_options()
    function_to_call, response_text = Application.get_selected_menu_item()
    Application.call_function_get_response(function_to_call, response_text)

Choose a menu
1 Get total number of samples
2 Get Name for an given Accession
3 Get Filtered Accessions for an given Attribute and Value
4 Examples of such queries and results
5 Exit
Option: 5


SystemExit: 

/Users/okgarces/Documents/Alkanza/virtualenvs/alkanzaprojections/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [115]:
a = "  asd "
b = a.isspace()
print(b)

False


In [123]:
Api_client.get()

5795298